# Test Colab stuff 

In [1]:
%load_ext autoreload
%autoreload

In [2]:
from tests import TEST_NB, FAIL_NB
from nb_helpers.utils import *
from nb_helpers.wandb import *

In [3]:
summary_nbs(path=TEST_NB)

CONSOLE.is_terminal(): False


Writing output to summary_table.csv

Reading 1 notebooks

┌───┬───────────────┬────────────┬───────┬────────────────────────────────────────────────┬───────────┐
│ # │ fname         │ tracker    │ wandb │ python libs                                    │ colab_idx │
├───┼───────────────┼────────────┼───────┼────────────────────────────────────────────────┼───────────┤
│ 0 │ ]8;id=202440;https://colab.research.google.com/github/wandb/nb_helpers/blob/main/tests/data/test_nb.ipynb\test_nb.ipynb]8;;\ │ 0: tracker │       │ os, sys, logging, pathlib, fastcore, itertools │ 1         │
└───┴───────────────┴────────────┴───────┴────────────────────────────────────────────────┴───────────┘

The markdown file summary_table.md has been created!!!

The markdown file summary_table.md has been created!!!

Creating github issue via file: github_issue.md

## Search tracker and libs

In [4]:
from rich import print
from rich.console import Console

from nb_helpers.utils import *
from nb_helpers.run import *
from nb_helpers.wandb import *

In [5]:
repo = Path.cwd().parent
test_nb = repo/'tests/data/test_nb.ipynb'

In [6]:
fname = test_nb
notebook = read_nb(fname)
print(fname)

/Users/tcapelle/wandb/nb_helpers/tests/data/test_nb.ipynb

In [7]:
notebook["cells"][0]

{'cell_type': 'markdown',
 'metadata': {},
 'source': '# This is a Test NB\n<!--- @wandbcode{tracker} -->'}

In [8]:
git_local_repo(fname)

Path('/Users/tcapelle/wandb/nb_helpers')

In [9]:
get_colab_url(fname)

'https://colab.research.google.com/github/wandb/nb_helpers/blob/main/tests/data/test_nb.ipynb'

## Detect Pyton libs

In [10]:
import re

In [11]:
nb = read_nb(TEST_NB)

In [12]:
def search_cells(nb, string: str = None, cell_type=CellType.code):
    "Search string in notebook cells, you can pass comma separated strings"
    strings = ifnone(string, "").replace(" ", "").split(",")
    cells = L()
    for cell in nb["cells"]:
        if cell["cell_type"] == cell_type:
            if any([string in cell["source"] for string in strings]):
                cells.append(cell["source"])
    return cells

In [13]:
text_list = search_cells(nb, "import,from")
text_list

(#2) ['import os, sys\nimport logging as log\nfrom pathlib import Path\nfrom fastcore.basics import ifnone\n\ndef f(x): \n    return x','from itertools import chain']

In [14]:

text_list = L([x.split("\n") for x in text_list]).concat()
text_list = [line for line in text_list if (("from" in line) or ("import" in line))]
text_list

['import os, sys',
 'import logging as log',
 'from pathlib import Path',
 'from fastcore.basics import ifnone',
 'from itertools import chain']

In [15]:
strings = ['import tensorflow as tf','import tf.keras as K','import numpy','import sys, os']

In [16]:
after_import_regex = re.compile(r'^import\s([^\.]*)', re.VERBOSE)
[after_import_regex.search(input).group(1) for input in strings]

['tensorflow as tf', 'tf', 'numpy', 'sys, os']

In [17]:
before_as_regex = re.compile(r'([^\s]*?)\sas\s', re.VERBOSE)
before_as_regex.search('tensorflow as tf').group(1)

'tensorflow'

In [18]:
between_from_import_regex = re.compile(r'^from\s(.*?)\simport', re.VERBOSE)
between_from_import_regex.search("from pathlib import Path").group(1)

'pathlib'

In [19]:
    strings = [
        "import tensorflow as tf",
        "import tf.keras as K",
        "import numpy",
        "import sys, os",
        "from fastcore.basics import ifnone",
    ]

In [20]:
def _extract_libs(strings):
    "Automatically detect libraries imported in `strings`"

    after_import_regex = re.compile(r'^import\s([^\.]*)', re.VERBOSE)
    before_as_regex= re.compile(r'([^\s]*?)\sas\s', re.VERBOSE)
    between_from_import_regex=re.compile(r'^from\s(.*?)\simport', re.VERBOSE)


    def _search_with_regex(regex, string):
        res = regex.search(string)
        if res is not None:
            return res.group(1)
        else:
            return ""

    libs = []
    for string in strings:
        if "from" in string:
            string = _search_with_regex(between_from_import_regex, string).split(".")[0]
        else:
            string = _search_with_regex(after_import_regex, string)
            if "as" in string:
                string = _search_with_regex(before_as_regex, string)
        libs.append(string.replace(" ", "").split(","))
    return L(libs).concat()


In [21]:
_extract_libs(strings)

(#6) ['tensorflow','tf','numpy','sys','os','fastcore']

In [22]:
from nb_helpers.utils import extract_libs

In [23]:
extract_libs(strings)

(#6) ['tensorflow','tf','numpy','sys','os','fastcore']

In [24]:
def detect_imported_libs(notebook):
    text_list = search_cells(notebook, "import,from")

    # format lines
    text_list = L([x.split("\n") for x in text_list]).concat()
    text_list = [line for line in text_list if (("from" in line) or ("import" in line))]
    
    return _extract_libs(text_list)


In [25]:
fail_nb = read_nb(FAIL_NB)
detect_imported_libs(fail_nb)

(#0) []

In [26]:
detect_imported_libs(read_nb(TEST_NB))


(#6) ['os','sys','logging','pathlib','fastcore','itertools']

# Colab badge/meta

In [32]:
from nb_helpers.colab import *

In [33]:
nb = read_nb(TEST_NB)
add_colab_metadata(nb)

{'cells': [{'cell_type': 'markdown',
   'metadata': {},
   'source': '# This is a Test NB\n<!--- @wandbcode{tracker} -->'},
  {'cell_type': 'markdown',
   'metadata': {},
   'source': '<a href="https://colab.research.google.com/github/wandb/nb_helpers/blob/main/tests/data/test_nb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>'},
  {'cell_type': 'markdown',
   'metadata': {},
   'source': 'this notebooks has code and text and this fancy walala'},
  {'cell_type': 'code',
   'execution_count': None,
   'metadata': {},
   'outputs': [],
   'source': 'import os, sys\nimport logging as log\nfrom pathlib import Path\nfrom fastcore.basics import ifnone\n\ndef f(x): \n    return x'},
  {'cell_type': 'code',
   'execution_count': None,
   'metadata': {},
   'outputs': [],
   'source': 'f(2)'},
  {'cell_type': 'code',
   'execution_count': None,
   'metadata': {},
   'outputs': [],
   'source': 'with open("file.txt", "w") as w

In [34]:
write_nb(nb, TEST_NB)